# 0. 추천시스템의 이해

- 정의 : 사용자에게 상품을 제안하는 소프트웨어 도구이자 기술     
>**어떤 사용자(타겟)**에게 어떤 상품을 **어떻게 추천할지**에 대한 이해   

- 파레토의 법칙 : 상위 20%가 80%의 가치를 창출한다    
- 롱테일 법칙 : 하위 80%가 상위 20%의 가치보다 크다 ex) 아마존    


0. Apriori 연관분석
1. 넷플릭스 우승자의 solution : SVD를 이용한 협업필터링
2. 데이터를 어떻게 다룰지에 대한 논의 : Spark 발전
3. 딥러닝의 발전과 강화학습과 결합 > 초 개인화 추천시스템

-----

# 0-1. 연관분석(Association)
- 정의 : 상품과 상품사이에 어떤 **연관**이 있는지 찾아내는 알고리즘(장바구니 분석)
> 연관 : 얼마나 같이 구매되는가?     
A아이템을 구매하는 사람이 B아이템을 구매하는가?     

ex) 월마트에서 맥주를 구매할 때 기저귀를 같이 구매하는 경향이 크다는 것을 밝혀내 진열하는 전략을 세우기도 하였음. 



- 규칙 생성 : **가능한 아이템의 모든 경우의 수**를 나열하고 탐색해 지지도, 신뢰도, 향상도가 높은 규칙을 찾아내는 방식(순열과 조합 방식)       


- 문제점 : **아이템의 증가**에 따른 **규칙에 수가 기하급수적으로 증가**
> Apriori 알고리즘 등장

-----
### Apriori 알고리즘 
빈번한 아이템셋은 하위 아이템셋 또한 빈번하므로, 빈번하지 않은 아이템셋은 하위 아이템셋 또한 빈번하지 않을 것이다


확률적으로 2,3을 같이 구매하는 경우의 수보다 0,2,3을 같이 구매하는 경우의수가 더 클 수밖에 없음. 그럼 2,3 같이 구매하는 경우의수가 적다면 그냥 아래꺼까지 제거해주자

#### < 알고리즘 >
1. 단일 항목집단(하나씩 떨어져있는 부분)에서 **최소 지지도 이상의 항목만 선택**   
2. 1에서 선택된 집단에서 두개 항목집단 생성

#### < 예시 >     
implicit data
1. 구매 이력을 Sparse matrix로 변형(비암시적 피드백(implicit feedback) : 상품의 구매에 대한 사용자의 만족 여부를 모름. 영화의 평점 역할이 없음. 대부분의 데이터)      



2. 지지도 계산

ex) 0.75 : 4번 중 3번의 거래 발생. 주스는 제거됨!



3. 2에서 선택된 항목만을 대상으로 2개 항목집단 생성

4. 위의 과정을 k-item set을 생성할때까지 반복    

#### < 장점 >
- 원리가 간단하고 쉽게 이해할 수 있음(유의한 연관성을 갖는 구매패턴을 찾아줌)  


#### < 단점 >
- 데이터가 크면(item이 많으면) 속도가 느리고 연산량이 많음
- 실제 사용 시 많은 연관상품이 나타나는 단점이 있음(상관관계는 의미해도, 인과관계는 의미하지 않음)

> FP-Growth 알고리즘

-----


### FP-Growth 알고리즘       
최소 지지도 이상의 아이템을 선택하고, 트리를 접목    


지지도가 높은 순으로 정렬    
부모 노드에서 거래 순서대로 아이템들을 자식 노드에 추가     


조선부 패턴을 생성한다(양상추를 샀을 때 뭘 사는지?)

#### < 장점 >
- Apriori 알고리즘보다 빠르고 2번의 탐색만 필요로 함  
- 후보 item set을 생성할 필요가 없음

#### < 단점 >    
   


cf) 신규 고객의 추천?
  
신규 고객이 어떻게 정의되냐에 따라 다름
같이 진열을 하는건 cold start와는 관계가 없음. 아무것도 구매하지 않은 상황에서 치킨을 구매한다면, 치킨무를 같이 띄워준다던지 이런걸 같이할 수는 있을거임. 

----
## 실습

In [9]:
# !pip install mlxtend
import mlxtend   # apriori나 fp-growth 알고리즘
import numpy as np
import pandas as pd

In [4]:
data = np.array([
    ['우유', '기저귀', '쥬스'], # 0번의 거래
    ['양상추', '기저귀', '맥주'], # 1번의 거래
    ['우유', '양상추', '기저귀', '맥주'],
    ['양상추', '맥주']
])

In [5]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [6]:
%%time
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.5, use_colnames=True)

Wall time: 18.9 ms


,support,itemsets
0,0.75,(기저귀)
1,0.75,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,"(기저귀, 맥주)"
5,0.50,"(기저귀, 양상추)"
6,0.50,"(우유, 기저귀)"
7,0.75,"(맥주, 양상추)"
8,0.50,"(기저귀, 맥주, 양상추)"


In [7]:
# fp growth
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(data).transform(data)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,True,True,True,False
3,False,True,True,False,False


In [8]:
%%time
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.5, use_colnames=True)

Wall time: 4.99 ms


,support,itemsets
0,0.75,(기저귀)
1,0.50,(우유)
2,0.75,(양상추)
3,0.75,(맥주)
4,0.50,"(기저귀, 맥주)"
5,0.50,"(기저귀, 양상추)"
6,0.50,"(기저귀, 맥주, 양상추)"
7,0.50,"(우유, 기저귀)"
8,0.75,"(맥주, 양상추)"


# 1. 콘텐츠기반 모델
Item을 벡터로 표현한 후 유사도를 계산   


#### < 유사도의 종류 >   
상황에 따라 적용해야함. 종류가 매우 많음. 추천 결과도 유사도에 따라 달라짐
1. 유클리디안 유사도     
벡터간의 거리 단순 계산. but 두 벡터의 분포나 범위가 다를 경우 상관성을 놓침.



2. 코사인유사도      
둘이 얼마나 **비슷한 방향**을 가지는지? 에 대한 것!    
동일한 방향을 가지면 similarity가 1이 됨. 세타가 90도의 방향을 가지면 similarity가 0. 정 반대면 -1로 최소의 값을 가지게 됨.        
> **벡터의 크기가 중요하지 않은 경우**(빈도수가 중요한 경우. 길이가 고르지 않더라도 얼마나 나왔는지 비율을 계산하기 쉬움)에는 잘 작동. 유클리디안과 달리 벡터의 크기가 중요한경우 잘 작동 X


cf) 유클리디안 거리는 회색과 파랑색을 가깝기때문에 유사한 벡터로 찾음. 
but 코사인유사도는 방향에 따라 판단하므로 노랑색과 빨강색을 유사한 벡터로 인지할 것임. 



3. 피어슨 유사도(상관관계 분석)  

4. 자카드 유사도 : 집합에서 얼마나 결합된 부분이 있는지를 계산  

cf) 보통 유사도를 상황에 적절하게 맞게 여러가지를 조합해서 사용함!   
(유사도값을 적절히 가중치를 둬서 더해서 모델을 하나 만들거나, 각각 유사도를 다르게 해서 모델을 각각 만들어서 모델을 앙상블하거나)     

고객 집단에 따라 다른 추천시스템 모델을 적용할 수 있음!    


### Item을 벡터로 표현하는 법 
#### 자연어처리 알고리즘      
---
### 1. TF-IDF 

- 정의 : 단어 빈도(TF)와 전체 문서에서 특정 단어가 얼마나 등장하는지 (DF)를 통해 문서내 단어의 가중치를 계산하는 방법      
(조사나 관사등을 패널티를 주어 배제하기 위해)     

< 장점 > : 직관적인 해석이 가능   
< 단점 > : 대규모 말뭉치를 다룰 때 메모리상의 문제 발생(높은 차원, 매우 sparse)   

> word2vec


In [1]:
docs = [
  '먹고 싶은 사과', # 문서0 
  '먹고 싶은 바나나', # 문서1
  '길고 노란 바나나 바나나', # 문서2 
  '저는 과일이 좋아요' # 문서3 
]

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer() # Counter Vectorizer 객체 생성 
# 문장을 Counter Vectorizer 형태로 변형 
countvect = vect.fit_transform(docs) 
countvect

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [4]:
countvect.toarray()

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [5]:
vect.vocabulary_

{'먹고': 3,
 '싶은': 6,
 '사과': 5,
 '바나나': 4,
 '길고': 1,
 '노란': 2,
 '저는': 7,
 '과일이': 0,
 '좋아요': 8}

In [6]:
sorted(vect.vocabulary_)

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [8]:
import pandas as pd
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = ['문서1', '문서2', '문서3', '문서4']
countvect_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0,0,0,1,0,1,1,0,0
문서2,0,0,0,1,1,0,1,0,0
문서3,0,1,1,0,2,0,0,0,0
문서4,1,0,0,0,0,0,0,1,1


In [9]:
# 위의 Data Frame 형태의 유사도를 계산 
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 1.        , 0.47140452, 0.        ],
       [0.        , 0.47140452, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

In [10]:
# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfvect = vect.fit(docs)

tfidv_df = pd.DataFrame(tfvect.transform(docs).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = ['문서1', '문서2', '문서3', '문서4']
tfidv_df

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문서1,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
문서2,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
문서3,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
문서4,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidv_df, tfidv_df) # countvectorizer의 문제점을 해결

array([[1.        , 0.60784064, 0.        , 0.        ],
       [0.60784064, 1.        , 0.42980824, 0.        ],
       [0.        , 0.42980824, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

---

### 2. Word2Vec
단어를 의미기반으로 바꾸고, 배치기반의 딥러닝 학습을 하면 어떨까? 라는 생각    

- 추론기반의 방법 : 주변 단어(맥락)이 주어졌을 때 중간에 무슨 단어가 들어가는지 추측하는 작업  


- 정의 : 단어간 유사도(주변 단어의 유사함)를 반영하여 단어를 벡터로 바꿔주는 임베딩 방법론. sparse matrix의 단점을 해소하고 저차원 공간에 벡터로 매핑.    

" 비슷한 위치에 등장하는 단어들은 비슷한 의미를 가진다 " 


#### <알고리즘>   
- CBOW : 주변의 단어를 다지고 중간의 단어를 예측      
- Skip-Gram : 중간의 단어로 주변의 단어를 예측(CBOW보다 성능이 좋음)   

input이 원핫 벡터로 들어옴. soft-max 값을 크로스엔트로피로 loss 계산

#### < 장점 >    
- item feature를 사용하기 때문에, 추천이 된 이유를 설명하기 용이함.    
- cold start문제를 해결 가능(사용자가 평점을 매기지 않은 새로운 item이 들어올 경우에도 추천 가능)   

#### < 단점 >
- item feature를 정확히 추출해야함. 제대로 추출하지 못하면 정확도가 낮음.
- 도메인 지식이 필요함.
- 기존의 item과 유사한 item으로만 추천하기 때문에 새로운 item에 대한 추천이 어려움. 


# 2. 협업필터링  
- 정의 : 사용자의 구매 패턴이나 평점을 가지고 다른사람의 구매패턴, 평점을 통해 추천하는 방법.   

### 1) Neighborhood based method    
- User-based : 사용자의 구매패턴(평점)과 유사한 사용자를 찾음(유사한 사람을 찾음)
- Item-based : 특정 사용자가 준 점수간 유사한 상품을 찾음(유사한 상품을 찾음)

ex) Explicit Feedback
- 사용자마다 평점 스케일이 다름(전반적으로 짜게주는 경우 존재) bias로 삽입하기!

#### <장점>
- 간단하고 직관적임. 추천 이유를 정당화하기 쉬움
- item과 user가 추가되어도 안정적임

#### <단점>  
- 많이 보는 것 위주로밖에 추천이 될 수 밖에 없음
- 희소성때문에 제한된 범위가 있음(아무도 해리포터를 평가하지 않으면, 해리포터에 대한 등급 예측을 제공할 수 없음)

----
### 2) Latent Factor(잠재요인)    
- 정의 : rating matrix의 빈공간을 채우기 위해 사용자와 상품을 잘 표현하는 차원(latent factor)를 찾는 방법   
(user-item matrix를 행렬분해)

####  SGD
고윳값 분해와 같은 행렬을 대각화하는 방법. 유저와 아이템 동시에 최적화    

1. user latent와 item latent를 임의로 초기화(random initialize)       
2. 1번의 둘을 product한 후, 기존 rating행렬과 Gradient Descent진행하며 오차를 계산함.    
3. GD로 추측된 rating이 원래 rate와 비슷해지면, 이를 이용해 rating 추론   

#### < 장점 >
매우 유연한모델로 다른 loss fuction도 사용할 수 있음. 


----
### 3) ALS    
- 정의 : SGD가 두개의 행렬을 동시에 최적화하는 방법이라면, ALS는 두 행렬 중 **하나를 고정시키고 다른 행렬 하나를 최적화**하는 방법.    
기존의 최적화 문제가 convex 형태로 바뀌기에 수렴된 형태를 찾을 수 있음. 

< 알고리즘 > 
1. 초기 item, user 행렬 initialization
2. item 행렬을 고정하고 user 행렬을 최적화(선형대수처럼 답이 정해져 있음)
3. user 행렬을 고정하고 item 행렬을 최적화   


----
### 4) 협업필터링의 전반의 장단점
#### < 장점 > 
- 도메인 지식이 필요하지 않음(추가 문맥 정보가 필요 없음. baseline으로 잡음)

#### < 단점 > 
- 새로운 아이템에 다루기 힘듦. 
- side feature를 포함시키기 어려움(메타데이터)


# 3. 평가함수
- 정의 : 얼마나 잘 추천해주고 있는가?  

1) Accuracy   
- 내가 추천해준 영화를 고객이 봤나 vs 안봤나? (추천한 영화를 많이 볼수록 정확도 상승)
- but 추천하지 않은 영화의 수는 추천한 영화수에 비해 매우 많아서 편향될 수 있음. 


2) MAP
- 추천의 순서에 따라 값에 차이가 남. precision을 이용해 계산

3) NDCG 
- MAP를 보완한 것
